In [21]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [14]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(10000, replace=False)
# df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
9206,"one of the worst films I have EVER seen, but e...",neg
44484,"Bill Paxton, of Aliens, Near Dark, and Termina...",pos
4991,Oh If any day u wanna see a supernatural thril...,neg
9407,"If I had never seen the first Road House, then...",neg
33683,I watched this movie after having so much of t...,neg
...,...,...
33406,A film like Amazing Grace and Chuck is a perfe...,neg
2960,I liked the understated character that Laura L...,neg
8158,If you make a suspense movie it is kind of imp...,neg
23479,Abbott and Costello's talents shine in the hap...,pos


### Palabras de parada

In [15]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself'

### Normalizar el texto

In [16]:
#pip install pycontractions

In [17]:
import re

def limpiar_html(raw_html):
    reg_limpiar = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    texto_limpio = re.sub(reg_limpiar, '', raw_html)
    return texto_limpio

def normalizar_critica(critica):
    critica_sin_html = limpiar_html(critica)
    tokens = nltk.tokenize.casual.casual_tokenize(critica_sin_html, "english")
    tokens_normalizados = [token.lower() for token in tokens if token not in palabras_de_parada_ingles]
    return " ".join(tokens_normalizados)

def normalizar_fila(fila):
    nueva_fila = fila
    critica_normalizada = normalizar_critica(fila['Review'])
    nueva_fila['Review'] = critica_normalizada
    return nueva_fila

In [18]:
df_criticas = df_criticas.apply(normalizar_fila, axis=1)
# df_criticas_normalizadas
df_criticas

,Review,Label
9206,one worst films i ever seen extremely funny pu...,neg
44484,bill paxton aliens near dark terminator fame s...,pos
4991,oh if day u wanna see supernatural thriller tu...,neg
9407,if i never seen first road house i guess movie...,neg
33683,i watched movie much trouble downloading rapid...,neg
...,...,...
33406,a film like amazing grace chuck perfect exampl...,neg
2960,i liked understated character laura linney pla...,neg
8158,if make suspense movie kind important villain ...,neg
23479,abbott costello's talents shine happily childi...,pos


### Obtener el vocabulario del problema

In [19]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [20]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
#vocabulario_problema.keys()

### One Hot Encoding

In [21]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [22]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [23]:
# Eliminar el dataframe de la memoria RAM
lst = [df_criticas]
del df_criticas
del lst

In [ ]:
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas_normalizadas['Label'].ravel()

In [ ]:
# Eliminar la matriz one_hots de la memoria RAM
del one_hots

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [33]:
from sklearn.linear_model import LogisticRegression
clasificador_reg_log = LogisticRegression(random_state=0, solver='liblinear') #Falta añadir el solver del magister...
clasificador_reg_log.fit(X_train, Y_train)
print('accuracy del clasificador: {0:.2f}'.format(clasificador_reg_log.score(X_test, Y_test)))

accuracy del clasificador: 1.00


### Guardar el clasificador como un archivo

In [34]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [35]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [36]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()